In [ ]:
import numpy as np
from scipy.optimize import minimize

In [ ]:
#Need a script that calculates percent of vehicles in area that are EVs based on vehicle registration data
#Defines variable percentEV

In [ ]:
def chargingDemand(demandHourly,percentEV,chargingProbability,chargers):
    #Takes inputs: array demandHourly which is 13 (hours per day) by 7 (days per week), float percentEV, float chargingProbability
    #Returns total number of charging instances per week weeklyChargeCount

In [ ]:
def profit(installCost,rebatePercent,chargeCharge, weeks, demandHourly,percentEV,chargingProbability,chargers):
    #Takes inputs: float installCost in CAD, float rebatePercent, chargeCharge in CAD/hour (change this eventually to CAD/kWh and weeklyChargeCount to kwH/week)
    #plus params for ChargingDemand
    weeklyChargeCount = chargingDemand(demandHourly,percentEV,chargingProbability,chargers)
    costs = installCost*rebatePercent
    #This needs to be adjusted for inflation later
    revenue = weeklyChargeCount*chargeCharge*weeks
    return(revenue-costs)
    #Returns total profit/loss in CAD

In [ ]:
#Lifecycle analysis later

In [ ]:
#Battery modelling later (this might go in the script up top actually)

In [ ]:
def chargerSizing(demandHourly,percentEV,chargingProbability):
    #This is an optimization function that finds the optimal number of chargers to maximize profit
    defaultChargers = 1
    result = minimize(lambda x: -profit(demandHourly,percentEV,chargingProbability,x), defaultChargers, method='BFGS')
    chargers = result.x[0]
    profit = -result.fun
    print("The optimal number of chargers is")
    print(chargers)
    print("The profit will be")
    print(profit)
    return(chargers)